BWA Hands on.

For this class we will be using BWA to perform alignments on genomes. In your projects you should build an index depending on what your project, is SARS, HUman, mouse, etc....

As Usually, we will start by installing the required programs:

In [ ]:
#new programs!
!apt install bwa
!apt install samtools
!apt install bcftools

### Old programs, can you identify them?
!sudo apt-get install -y default-jre
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
!unzip fastqc_v0.11.9.zip
!chmod +x FastQC/fastqc
!pip install cutadapt


Download some datasets

In [ ]:
!gdown 1J6nku7fSKE5t9yGgw0m1sUGy6regdmeb
!gdown 1e05YYPAHcKEjNYgwMntknvKY-Je-W2bh

For today class to make everything go faster I will focus on a small genome, for your project you should use your specific genome of interest, the slides list the places where you can donwload these, remember you can use `wget`, or `curl`.

In [ ]:
!gdown 1yTOH-9w0RbcBhwetNWzYxFXENtiyPlzW

Downloading...
From: https://drive.google.com/uc?id=1yTOH-9w0RbcBhwetNWzYxFXENtiyPlzW
To: /content/WuhanRef.fasta
100% 30.4k/30.4k [00:00<00:00, 67.4MB/s]


Let's see whats different after downloading these datasets: `ls`

In [ ]:
!more WuhanRef.fasta
###### Type !ls you should see three new files, corresponding to pair end reads and the wuhan reference genome.\

In [ ]:
!bwa index -p Wuhan WuhanRef.fasta


[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.01 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index -p Wuhan WuhanRef.fasta
[main] Real time: 0.052 sec; CPU: 0.015 sec


In [ ]:
!ls

drive	fastqc_v0.11.9.zip  SARS.R1.fastq  Wuhan.amb  Wuhan.bwt  WuhanRef.fasta
FastQC	sample_data	    SARS.R2.fastq  Wuhan.ann  Wuhan.pac  Wuhan.sa


In [ ]:
!more SARS.R1.fastq


Run bwa and see what happens, it will likely print help, refer to the manual https://bio-bwa.sourceforge.net/bwa.shtml if you want to know more.
HINT : `bwa`

Build an index with your reference genome.` !bwa index ` run it without commands first, to print out help.

After you actually build it, run `!ls` to see whats different

#WARNING

here you have two options run bwa aln or mem. Options are discussed elsewhere and there is no clear consensus on which one is best. For now in our class, we will use `!bwa aln` `!bwa sampe `for smaller genomes and short reads. And we will use `!bwa mem `for human or mouse genomes.

In [ ]:
#### Option Number 1 bwa mem
#### !bwa mem WuhanRef SARS.R1.fastq SARS.R2.fastq > SARS.sam


In [ ]:
#### Option Number 2 bwa aln
!bwa aln Wuhan SARS.R1.fastq > SARS.R1.sai
!bwa aln Wuhan SARS.R2.fastq > SARS.R2.sai
!bwa sampe Wuhan SARS.R1.sai SARS.R2.sai  SARS.R1.fastq SARS.R2.fastq > SARS.bam

In [ ]:
!head -n 100 SARS.bam

In [ ]:
!more drive/MyDrive/MATH5300/insurance_test.csv


age,sex,bmi,children,smoker,region,charges
25,female,34.485,0,no,northwest,3021.80915
42,female,40.37,2,yes,southeast,43896.3763
20,male,31.13,2,no,southeast,2566.4707
18,male,33.535,0,yes,northeast,34617.84065
38,male,28.025,1,no,northeast,6067.12675
39,female,32.5,1,no,southwest,6238.298
54,male,39.6,1,no,southwest,10450.552
54,female,21.47,3,no,northwest,12475.3513
41,male,30.78,3,yes,northeast,39597.4072
46,female,30.2,2,no,southwest,8825.086
58,male,35.7,0,no,southwest,11362.755
28,male,23.8,2,no,southwest,3847.674
63,male,30.8,0,no,southwest,13390.559
48,male,32.3,1,no,northwest,8765.249
53,female,37.43,1,no,northwest,10959.6947
32,female,33.155,3,no,northwest,6128.79745
45,male,22.895,2,yes,northwest,21098.55405
62,male,31.46,1,no,southeast,27000.98473
51,female,39.5,1,no,southwest,9880.068
44,female,25,1,no,southwest,7623.518
34,male,32.8,1,no,southwest,14358.36437
59,female,36.765,1,yes,northeast,47896.79135


# NOTES

Depending on what you run, you might have a sam or a bam file. You can convert between both using samtools and you can chose which output to spit, not shown but I encourage you to read the manual. This is all you are going to need for your project.

### Quick brain teaser:
- How many reads are mapped to the SARS genome?
- How many reads are in our original fastqs?  
- What do you think are the rest of the reads?
- Look at the coordinates of the alignments. What do you see?

In [ ]:
!grep 'NC_045512.2' SARS.bam | wc

1305821 27282190 373786200


In [ ]:
!head -n 1000 SARS.bam

In [ ]:
####### Ignore this, this is a note for me, it is a one liner that modifies the headers from reads downloaded from SRR
####### !sed 's/\(\.[0-9]*\)\.[0-9]*/\1/g' SARS.R2.fastq > SARS.R2.S.fastq

Introduction to Variant Calling, we will be using  bcftools to call variants based on our aligned reads. The very first thing we need to do is sort the alignment, then we will be using pileup, which

In [ ]:
!samtools view -bS -q 12 SARS.bam | samtools sort -o SARS.bam.sorted


In [ ]:
!samtools view SARS.bam.sorted

In [ ]:
!bcftools mpileup -f WuhanRef.fasta --max-depth  2000 SARS.bam.sorted  | bcftools call --multiallelic-caller --variants-only --ploidy 1  -mv -Oz -o SARS.Variants.vcf.gz

[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 2000


Explore the VCF, what do you see? How many mutations are detected?

In [ ]:
!gzip -df SARS.Variants.vcf.gz


In [ ]:
!cat SARS.Variants.vcf

In [ ]:
####### This is to get you guys excited about next class/(es)
!pip install cyvcf2
#!gzip -df SARS.Variants.vcf.gz
vcf_path = 'SARS.Variants.vcf'

In [ ]:
import cyvcf2

# Create a VCF reader object
vcf_reader = cyvcf2.VCF(vcf_path)

with open('output.tsv', 'w') as file:
# Loop through each variant in the VCF file
    for variant in vcf_reader:
    # Access various properties of the variant
    ### Se note below
        genotype = variant.gt_types[0]
        data_string = f"{variant.CHROM}\t{variant.POS}\t{variant.ID}\t{variant.REF}\t{','.join(variant.ALT)}\t{variant.QUAL}\t{variant.FILTER}\t{genotype}\n"
        # Write the data string to the file
        file.write(data_string)

In [ ]:
!more output.tsv

NC_045512.2	21	None	C	T	43.41469955444336	None	3
NC_045512.2	241	None	C	T	228.42799377441406	None	3
NC_045512.2	1387	None	C	T	225.4219970703125	None	3
NC_045512.2	2832	None	A	G	228.4290008544922	None	3
NC_045512.2	3037	None	C	T	225.41700744628906	None	3
NC_045512.2	5386	None	T	G	228.42599487304688	None	3
NC_045512.2	8393	None	G	A	225.41700744628906	None	3
NC_045512.2	10029	None	C	T	225.42599487304688	None	3
NC_045512.2	10449	None	C	A	228.427001953125	None	3
NC_045512.2	11288	None	T	A	6.382820129394531	None	3
NC_045512.2	11291	None	G	A	226.42100524902344	None	3
NC_045512.2	11537	None	A	G	225.42599487304688	None	3
NC_045512.2	13195	None	T	C	228.43499755859375	None	3
NC_045512.2	14408	None	C	T	225.41700744628906	None	3
NC_045512.2	15240	None	C	T	228.4290008544922	None	3
NC_045512.2	18163	None	A	G	225.41700744628906	None	3
NC_045512.2	21762	None	C	T	67.41609954833984	None	3
NC_045512.2	21766	None	A	C	11.15429973602295	None	3
NC_045512.2	21846	None	C	T	225.4219970703125	None	3
NC_045512.2	2

In [ ]:
import pandas

In [ ]:
columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'GENOTYPE']
df = pandas.read_csv("output.tsv", delimiter = "\t", header=None, names=columns)

In [ ]:
df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,GENOTYPE
0,NC_045512.2,21,None,C,T,43.414700,None,3
1,NC_045512.2,241,None,C,T,228.427994,None,3
2,NC_045512.2,1387,None,C,T,225.421997,None,3
3,NC_045512.2,2832,None,A,G,228.429001,None,3
4,NC_045512.2,3037,None,C,T,225.417007,None,3
5,NC_045512.2,5386,None,T,G,228.425995,None,3
6,NC_045512.2,8393,None,G,A,225.417007,None,3
7,NC_045512.2,10029,None,C,T,225.425995,None,3
8,NC_045512.2,10449,None,C,A,228.427002,None,3
9,NC_045512.2,11288,None,T,A,6.382820,None,3


In [ ]:
df_AG = df[(df['REF'] == 'A') & (df['ALT'] == 'G')]
df_CT = df[(df['REF'] == 'C') & (df['ALT'] == 'T')]
df_GA
df_TC


transition_part = len(df_AG) + len(df_CT)

8

In [ ]:
subset = df[df['REF'] == 'A']

subset

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,GENOTYPE
3,NC_045512.2,2832,None,A,G,228.429001,None,3
11,NC_045512.2,11537,None,A,G,225.425995,None,3
15,NC_045512.2,18163,None,A,G,225.417007,None,3
17,NC_045512.2,21766,None,A,C,11.154300,None,3
28,NC_045512.2,23013,None,A,C,225.425995,None,3
29,NC_045512.2,23040,None,A,G,225.421997,None,3
31,NC_045512.2,23055,None,A,G,228.427002,None,3
32,NC_045512.2,23063,None,A,T,225.421997,None,3
35,NC_045512.2,23403,None,A,G,228.436005,None,3
42,NC_045512.2,24424,None,A,T,228.425995,None,3
